<a href="https://colab.research.google.com/github/thetrotfreak/MCA572/blob/main/2347111_NNDL_P7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Imports

In [1]:
import re
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from tensorflow.keras import layers
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dropout, Dense

## Dataset Preparation:

In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("tgdivy/poetry-foundation-poems")

print("Path to dataset files:", path)

100%|██████████| 8.88M/8.88M [00:00<00:00, 193MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/tgdivy/poetry-foundation-poems/versions/1


### Import Dataset

In [3]:
from pathlib import PurePath

dataset = pd.read_csv(PurePath(path).joinpath("PoetryFoundationData.csv"))
dataset = dataset.drop_duplicates(subset=['Poem'])
dataset.head()

,Unnamed: 0,Title,Poem,Poet,Tags
0,0,\r\r\n Objects Used to Prop...,"\r\r\nDog bone, stapler,\r\r\ncribbage board, ...",Michelle Menting,NaN
1,1,\r\r\n The New Church\r\r\n...,"\r\r\nThe old cupola glinted above the clouds,...",Lucia Cherciu,NaN
2,2,\r\r\n Look for Me\r\r\n ...,\r\r\nLook for me under the hood\r\r\nof that ...,Ted Kooser,NaN
3,3,\r\r\n Wild Life\r\r\n ...,"\r\r\nBehind the silo, the Mother Rabbit\r\r\n...",Grace Cavalieri,NaN
4,4,\r\r\n Umbrella\r\r\n ...,\r\r\nWhen I push your button\r\r\nyou fly off...,Connie Wanek,NaN


### Dataset Columns

In [4]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13754 entries, 0 to 13833
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  13754 non-null  int64 
 1   Title       13754 non-null  object
 2   Poem        13754 non-null  object
 3   Poet        13754 non-null  object
 4   Tags        12854 non-null  object
dtypes: int64(1), object(4)
memory usage: 644.7+ KB


#### A Corpus of Poems

In [5]:
df_cleaned = dataset.dropna(subset=['Tags'])
df_cleaned = df_cleaned.drop_duplicates(subset=['Poem'])

corpus = "\n".join(df_cleaned['Poem'].head(100).values)

## Data Preprocessing:

### Lowercase and Strip

In [6]:
cleaned_text = corpus.lower()

cleaned_text = re.sub(r'[^a-zA-Z\s,\'"-.]', '', cleaned_text)
cleaned_text = re.sub(r'\r\r\n\n\r\r\n', '\n', cleaned_text)
cleaned_text = re.sub(r'[\r\r]', '', cleaned_text)
cleaned_text = re.sub(r'\n \n', '\n\n', cleaned_text)

cleaned_text = re.sub(r'\n{2,}', '\n\n', cleaned_text)
cleaned_text = re.sub(r' +', ' ', cleaned_text)
cleaned_text = cleaned_text.strip()

print(cleaned_text[:5000])

corpus = cleaned_text

invisible fish swim this ghost ocean now described by waves of sand, by water-worn rock. soon the fish will learn to walk. then humans will come ashore and paint dreams on the dying stone. then later, much later, the ocean floor will be punctuated by chevy trucks, carrying the dreamers decendants, who are going to the store.
dont bother the earth spirit who lives here. she is working on a story. it is the oldest story in the world and it is delicate, changing. if she sees you watching she will invite you in for coffee, give you warm bread, and you will be obligated to stay and listen. but this is no ordinary story. you will have to endure earthquakes, lightning, the deaths of all those you love, the most blinding beauty. its a story so compelling you may never want to leave this is how she traps you. see that stone finger over there that is the only one who ever escaped.
hour in which i consider hydrangea, a salt or sand plant, varietal, the question of varietals, the diet of every mot

In [7]:
paragraphs = corpus.split("\n\n")

# Now, split each paragraph into lines (split by single \n)
lines_in_paragraphs = [paragraph.split("\n") for paragraph in paragraphs]

temp_lines_in_paragraphs = []

lines = 0
for para in lines_in_paragraphs:
    temp = []
    for line in para:
      line = line.strip()
      if line == "":
        continue
      temp.append(line)
    if len(temp) > 0:
      temp_lines_in_paragraphs.append(temp)

    lines += len(para)

print(f"Paragraphs: {len(paragraphs)}")
print(f"Lines: {lines}")

Paragraphs: 612
Lines: 2790


### Tokenizing

In [8]:
flattened_lines = [line for paragraph in temp_lines_in_paragraphs for line in paragraph]

# Tokenize the corpus
tokenizer = Tokenizer()
tokenizer.fit_on_texts(flattened_lines)

# Get the vocabulary size
vocab_size = len(tokenizer.word_index) + 1  # Add 1 for padding token
print("Vocabulary size:", vocab_size)

# Convert the corpus into a sequence of integers (tokens)
sequences = tokenizer.texts_to_sequences(flattened_lines)
print(sequences[:20])  # Print the first 20 tokenized words

Vocabulary size: 4685
[[528, 370, 529, 24, 701, 163, 62, 1013, 36, 530, 3, 371, 36, 121, 1014, 702, 703, 1, 370, 25, 704, 4, 222, 82, 705, 25, 122, 1702, 6, 531, 284, 12, 1, 317, 706, 82, 223, 176, 223, 1, 163, 199, 25, 32, 1015, 36, 1703, 1016, 1704, 1, 1705, 1706, 47, 50, 106, 4, 1, 532], [107, 1707, 1, 135, 1708, 47, 247, 98, 33, 10, 1017, 12, 2, 177, 11, 10, 1, 1709, 177, 5, 1, 99, 6, 11, 10, 1710, 1711, 45, 33, 1712, 9, 533, 33, 25, 1713, 9, 5, 17, 318, 319, 9, 372, 1018, 6, 9, 25, 32, 1714, 4, 430, 6, 373, 30, 24, 10, 46, 1019, 177, 9, 25, 40, 4, 1715, 1716, 534, 1, 707, 3, 37, 224, 9, 123, 1, 374, 1717, 285, 35, 2, 177, 48, 1718, 9, 248, 83, 124, 4, 286, 24, 10, 53, 33, 1719, 9, 85, 13, 706, 708, 90, 56, 13, 10, 1, 94, 31, 47, 200, 1020], [535, 5, 125, 7, 1021, 709, 2, 710, 28, 371, 711, 1720, 1, 536, 3, 1721, 1, 1722, 3, 136, 80, 7, 100, 1022, 537, 14, 7, 40, 154, 7, 40, 154, 201, 2, 431, 3, 8, 137, 712, 285, 538, 1723, 7, 1724, 285, 285, 1023, 12, 1, 713, 3, 1725, 1, 249, 1726

### Sliding Window

In [9]:
sequence_length = 5  # Length of the sequence

# Create input-output pairs (X, y) based on the sliding window
X, y = [], []

for line in sequences:
    for i in range(sequence_length, len(line)):
        X.append(line[i-sequence_length:i])
        y.append(line[i])

print("X shape:", len(X))
print("y shape:", len(y))

X shape: 7098
y shape: 7098


### Padding

In [10]:
X = pad_sequences(X, maxlen = sequence_length, padding='pre')
y = np.array(y)
print("Padded X shape:", X.shape)

Padded X shape: (7098, 5)


## LSTM Model Development

In [11]:
# Define the LSTM model
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=100, input_length=X.shape[1]))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dropout(0.2))  # Dropout layer to prevent overfitting
model.add(Dense(vocab_size, activation='softmax'))  # Output layer for word prediction

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 5, 100)            468500    
                                                                 
 lstm (LSTM)                 (None, 5, 100)            80400     
                                                                 
 lstm_1 (LSTM)               (None, 100)               80400     
                                                                 
 dropout (Dropout)           (None, 100)               0         
                                                                 
 dense (Dense)               (None, 4685)              473185    
                                                                 
Total params: 1102485 (4.21 MB)
Trainable params: 1102485 (4.21 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


## Training:  

### Loss and Accuracy

In [12]:
from keras.utils import to_categorical

# One-hot encode the target labels
y = to_categorical(y, num_classes=vocab_size)
print("y shape after one-hot encoding:", y.shape)

y shape after one-hot encoding: (7098, 4685)


### Training

In [13]:
# Train the model
model.fit(X, y, epochs=10, batch_size=64)

Epoch 1/10
111/111 [==============================] - 5s 18ms/step - loss: 7.5349 - accuracy: 0.0595
Epoch 2/10
111/111 [==============================] - 2s 17ms/step - loss: 6.7245 - accuracy: 0.0609
Epoch 3/10
111/111 [==============================] - 2s 17ms/step - loss: 6.6110 - accuracy: 0.0609
Epoch 4/10
111/111 [==============================] - 2s 17ms/step - loss: 6.5455 - accuracy: 0.0609
Epoch 5/10
111/111 [==============================] - 2s 17ms/step - loss: 6.4702 - accuracy: 0.0609
Epoch 6/10
111/111 [==============================] - 2s 17ms/step - loss: 6.3438 - accuracy: 0.0609
Epoch 7/10
111/111 [==============================] - 2s 17ms/step - loss: 6.2469 - accuracy: 0.0617
Epoch 8/10
111/111 [==============================] - 2s 18ms/step - loss: 6.1558 - accuracy: 0.0682
Epoch 9/10
111/111 [==============================] - 2s 18ms/step - loss: 6.0355 - accuracy: 0.0734
Epoch 10/10
111/111 [==============================] - 2s 17ms/step - loss: 5.9057 - accura

## Text Generation:

In [14]:
def generate_text(seed_text, next_words, model, tokenizer, max_sequence_len, temperature=1.0, words_per_set=5):
    previous_word = ""  # Track the previous word to avoid consecutive repetition
    word_count = 0  # Track the number of words generated in the current set
    for i in range(next_words):
        # Tokenize the seed text
        tokenized_text = tokenizer.texts_to_sequences([seed_text])[0]

        # Pad the tokenized sequence
        tokenized_text = pad_sequences([tokenized_text], maxlen=max_sequence_len, padding='pre')

        # Predict the next word probabilities
        predicted = model.predict(tokenized_text, verbose=0)[0]

        # Apply temperature to predictions (to make them more diverse)
        predicted = np.log(predicted + 1e-7) / temperature
        predicted = np.exp(predicted) / np.sum(np.exp(predicted))  # Normalize to get valid probabilities

        # Sample the next word based on the adjusted probabilities
        predicted_word_idx = np.random.choice(len(predicted), p=predicted)
        predicted_word = tokenizer.index_word.get(predicted_word_idx, '')

        # Ensure the predicted word is not the same as the previous word
        if predicted_word == previous_word:
            continue  # Skip the word if it's the same as the previous one

        if word_count != 0:
          seed_text += " "

        # Append the predicted word to the seed text
        seed_text += predicted_word

        # Update the previous word
        previous_word = predicted_word

        # Track the number of words in the current set
        word_count += 1

        # If we've reached the words_per_set limit, add a comma and reset word count
        if word_count >= words_per_set:
            seed_text += ",\n"
            word_count = 0  # Reset the word count for the next set of words

    return seed_text

seed = "once upon a time "
generated_poem = generate_text(seed, next_words=50, model=model, tokenizer=tokenizer, max_sequence_len=X.shape[1], temperature=0.7, words_per_set=5)
print(generated_poem)

once upon a time of like before of the,
ritual that brothers breast reading,
of how darling how on,
the loving id her shape,
of the bank hid oranges,
his my tiger with sleek,
home hot emperor brother weekly,
machan weekly response applause the,
only of be moon stage,
i this with of sinker,



## Evaluation and Experimentation

In [15]:
# Define the LSTM model
model2 = Sequential()
model2.add(Embedding(input_dim=vocab_size, output_dim=100, input_length=X.shape[1]))
model2.add(LSTM(50, return_sequences=True))
model2.add(LSTM(50))
model2.add(Dropout(0.4))  # Dropout layer to prevent overfitting
model2.add(Dense(vocab_size, activation='softmax'))  # Output layer for word prediction

# Compile the model
model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 5, 100)            468500    
                                                                 
 lstm_2 (LSTM)               (None, 5, 50)             30200     
                                                                 
 lstm_3 (LSTM)               (None, 50)                20200     
                                                                 
 dropout_1 (Dropout)         (None, 50)                0         
                                                                 
 dense_1 (Dense)             (None, 4685)              238935    
                                                                 
Total params: 757835 (2.89 MB)
Trainable params: 757835 (2.89 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [16]:
# Train the model
model2.fit(X, y, epochs=30, batch_size=64)

Epoch 1/30
111/111 [==============================] - 4s 13ms/step - loss: 7.6633 - accuracy: 0.0568
Epoch 2/30
111/111 [==============================] - 1s 13ms/step - loss: 6.7770 - accuracy: 0.0609
Epoch 3/30
111/111 [==============================] - 1s 13ms/step - loss: 6.6687 - accuracy: 0.0609
Epoch 4/30
111/111 [==============================] - 1s 13ms/step - loss: 6.5874 - accuracy: 0.0609
Epoch 5/30
111/111 [==============================] - 1s 13ms/step - loss: 6.4976 - accuracy: 0.0609
Epoch 6/30
111/111 [==============================] - 1s 13ms/step - loss: 6.3771 - accuracy: 0.0609
Epoch 7/30
111/111 [==============================] - 1s 13ms/step - loss: 6.2976 - accuracy: 0.0610
Epoch 8/30
111/111 [==============================] - 1s 12ms/step - loss: 6.2421 - accuracy: 0.0607
Epoch 9/30
111/111 [==============================] - 1s 12ms/step - loss: 6.1963 - accuracy: 0.0614
Epoch 10/30
111/111 [==============================] - 1s 13ms/step - loss: 6.1549 - accura

In [20]:
seed = "A wild tale"
generated_poem = generate_text(seed, next_words=50, model=model2, tokenizer=tokenizer, max_sequence_len=X.shape[1], temperature=0.7, words_per_set=7)
print(generated_poem)

A wild taleeach club now it america become beds,
was the tribe of this long or,
pretend flawed and all as massing the,
weapon of insects of kob radiation cant,
were knives therapies moves there you checkbooks,
leading on the planet of the mouth,
the prayer and warm forever in the,

